In [9]:
import psycopg2 as pg2
import pandas as pd
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.max_colwidth', -1)
import pprint 
import string
import unicodedata
import numpy as np
import pickle

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import decomposition

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams
from nltk import pos_tag
from nltk import RegexpParser
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

## Import pickled cleaned data from previous project

In [10]:
with open('../data/boardgamedata.pickle', 'rb') as f:
    worddata, numdata = pickle.load(f)
    
worddata.reset_index(drop=True, inplace=True)

In [11]:
worddata[10:15]

game.id              name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Build LDA Model

In [12]:
#define common words in data to remove
#built from pervious project
sw = set(stopwords.words('english'))

COMMON_WORDS = ["game","point","board","player","rule","turn",
                "tile","card","deck","hand","points","victory",
                "win","lose","defeat","quot","players","play"
               ]
sw.update(COMMON_WORDS)

#### Remove numbers from vectorizer

Implement preprocess function

In [13]:
def preprocess_text(text):
    
#if text is empty, return empty bag of words
    if (text == None):
        return []

    if (len(text) < 1):
        return []
#remove numbers
    text = text.lower()
    text = re.sub(r'\d+', '', text)

#define common words in data to remove
    COMMON_WORDS = ["game","point","board","player","rule","turn",
                "tile","card","deck","hand","points","victory",
                "win","lose","defeat","quot","players","play"]
    
#removes common words
    for word in text.split():
        if word in COMMON_WORDS:
            text = text.replace(word, "")

#tokenize sentences and add part of speech tag
    sent_tokens = sent_tokenize(text)

    tokens = list(map(word_tokenize, sent_tokens))

    sent_tags = list(map(pos_tag, tokens))

    grammar = r"""
        SENT: {<(J|N).*>}    """

    cp = RegexpParser(grammar)
    ret_tokens = list()
    stemmer_snowball = SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer()

    for sent in sent_tags:
        tree = cp.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'SENT':
                t_tokenlist = [tpos[0].lower() for tpos in subtree.leaves()]
                
                t_tokens_lemmatize = list(map(lemmatizer.lemmatize, t_tokenlist))
                t_tokens_stemsnowball = list(map(stemmer_snowball.stem, t_tokens_lemmatize))
                ret_tokens.extend(t_tokens_stemsnowball)
    ret = []

#runs through the ret_tokens and compare with common words again and removes common words
    for token in ret_tokens:
        if token in COMMON_WORDS:
            pass
        else:
            ret.append(token)
    return(' '.join(ret))

In [15]:
count_vectorizer = CountVectorizer(stop_words=sw, 
                                   preprocessor=preprocess_text, 
                                   max_df=0.85,
                                  min_df=2, 
                                   max_features=1000,
                                  strip_accents='unicode')

In [ ]:
word_vec = count_vectorizer.fit_transform(worddata['description'])

/home/zy/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ourselv', 'sha', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [120]:
lda = LatentDirichletAllocation(learning_method='online', n_jobs=-2, 
                                random_state=1659)
lda.fit(word_vec)

LatentDirichletAllocation(learning_method='online', n_jobs=-2,
                          random_state=1659)

In [13]:
def top_topic_features(model, feature_names, num_features=10):
    sorted_topics = feature_names[model.components_.argsort(axis=1)[:, ::-1][:, :num_features]]
    return sorted_topics

In [121]:
feature_names = np.array(count_vectorizer.get_feature_names())
top_topic_features(lda, feature_names=feature_names)


array([['roll dice', 'highest score', 'six sid', 'number space',
        'action action', 'roll die', 'sid dice', 'number dice',
        'special action', 'six sid dice'],
       ['world war', 'end round', 'oppon rsquo', 'user summari',
        'war ii', 'world war ii', 'end winner', 'empti space',
        'head head', 'end er'],
       ['descript publish', 'special abil', 'fast pac', 'letter word',
        'trivia question', 'word letter', 'space space', 'word word',
        'first person', 'new one'],
       ['th centuri', 'dice dice', 'co op', 'first er', 'draw pile',
        'special power', 'next round', 'first expans', 'time time',
        'lsquo rsquo'],
       ['object first', 'civil war', 'properti rent', 'dis pile',
        'differ color', 'ultim goal', 'hous hotel', 'epsilon nu',
        'properti properti', 'lambda epsilon'],
       ['expans pack', 'finish line', 'piec piec', 'ldquo rdquo',
        'round round', 'back box', 'roll move', 'abstract strategi',
        'center

### Create Probability Matrix

In [122]:
def predict_proba(model, vectorizer, text):
    if type(text) == str:
        text = [text]
    vec_text = vectorizer.transform(text)
    doc_probs = model.transform(vec_text)
    return doc_probs

In [124]:
game_probs = predict_proba(lda, count_vectorizer, worddata.description)

KeyboardInterrupt: 

In [19]:
#pickle the fitted transformed model

with open('../models/lda.pickle', 'wb') as f:
    pickle.dump([lda,count_vectorizer,game_probs], f)

## Building the Recommender

In [22]:
# function to find index of FIRST  game given the game title

def find_game_idx(df, game_title):
    try:
        return df.name[df.name.str.contains(game_title)].index[0]
    except:
        return None

def find_game_id(df, idx):
    return df['game.id'].iloc[idx]


## TODO: if search returns multiple results, what then?? Need to solve this

In [23]:
search = worddata.name.str.contains('Betrayal')
search.sum()


11

## /TODO

In [25]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

In [26]:
def sort_by_distance(idx, probs, distance_funct=cosine_distances, num_games=10):
    #returns array of indices of games closest to the provided game index
    return distance_funct(probs[idx].reshape(1,-1), probs).ravel().argsort()[:10]

* Debug sort_by_distance function
* check distances inside the function


In [27]:
def find_closest_games_names(sorted_distances, names):
    #finds names of an array of game idx's
    name_array = names.iloc[sorted_distances]
    return name_array

In [34]:
game_index = find_game_idx(worddata,'Catan')
game_sim = sort_by_distance(game_index, game_probs)
find_closest_games_names(game_sim,worddata.name)


12                                          Catan
54254                              Pocket Paddles
71292                       Dice City: Shipwright
62950                          Small Star Empires
68682          Dice Bazaar: Monkey Mini Expansion
26389                         El Juego de la Liga
132                             Buy Low Sell High
5131                      The Ark of the Covenant
63732    All About Town: Columbia, South Carolina
2013                                      Wembley
Name: name, dtype: object

In [127]:
def recommend(df, game_title, prob_matrix, 
             distance_funct=cosine_distances, num_games=10):
    game_index = find_game_idx(df, game_title)
    game_sim = sort_by_distance(game_index, prob_matrix, distance_funct, num_games)
    game_recs = find_closest_games_names(game_sim, df.name)
    game_ids = find_game_id(df, game_rec.index)
    
#     rec_df = pd.DataFrame({'id':game_ids,'name':game_recs})
    rec_df = find_closest_games(game_sim,df)
    
    return rec_df

In [128]:
game_rec = recommend(worddata, 'Catan', game_probs)
game_rec

,game.id,name,description,category,mechanic
12,13,Catan,"In Catan (formerly The Settlers of Catan), pla...",[Negotiation],"[Dice Rolling, Hand Management, Modular Board,..."
54254,150834,Pocket Paddles,Pocket Paddles is a free print and play game f...,"[Nautical, Print & Play, Racing]","[Auction/Bidding, Point to Point Movement]"
71292,206255,Dice City: Shipwright,A promo card that was part of the Dice Tower 2...,"[City Building, Expansion for Base-game]","[Dice Rolling, Take That, Worker Placement]"
62950,178044,Small Star Empires,Small Star Empires is a quick area control gam...,"[Science Fiction, Territory Building]","[Area Control, Area Influence, Area Enclosure,..."
68682,199939,Dice Bazaar: Monkey Mini Expansion,Components Included: &#10;&#10; 4 Dice&#10...,"[Arabian, Card Game, Dice, Expansion for Base-...","[Dice Rolling, Press Your Luck, Set Collection..."
26389,35371,El Juego de la Liga,&quot;The Game of the League&quot; 2003/2004 b...,[Sports],"[Roll, Spin and Move]"
132,146,Buy Low Sell High,"Another of Reiner Knizia's fleet of games, thi...",[Economic],[Commodity Speculation]
5131,6779,The Ark of the Covenant,"A game based on Carcassonne, Ark of the Covena...","[Ancient, City Building, Religious]","[Area Control, Area Influence, Tile Placement]"
63732,180713,"All About Town: Columbia, South Carolina","The object of All About Town is for players, u...",[Educational],"[Roll, Spin and Move]"
2013,2721,Wembley,The game simulates the passage of English foot...,"[Dice, Sports]","[Dice Rolling, Simulation]"


In [126]:
def find_closest_games(sorted_distances, gamedata_df):
    #finds names of an array of game idx's
    name_array = gamedata_df.iloc[sorted_distances]
    return name_array

In [ ]:
data = pd.read_csv('recs.csv')

In [61]:
#save prob matrix for use later
with open('../models/prob_matrix.pickle', 'wb') as f:
    pickle.dump(game_probs,f)

## TODO: use suprise Library to test